### Construir dataset desde Txt

* *pasos* 

* 1- me fijo cómo identificar las películas vs usuarios
* 2- crear del len(df1) un vector con todas las movieids para poder concatenarlo
* 3- Para eso necesito tener una cantidad de iteraciones que corresponde a la cantidad de usuarios que puntuan una pelicula.

Es decir si tengo 

"volver al futuro":
* usuario 1: 5
* usuario 2: 6
* usuario 3: 7

voy a querer que al lado de cada uno de los usuarios diga "volver al futuro". Voy a necesitar contar que cantidad de usuarios hay y poder concatenar el id_movie por esas cantidades y crear un vector del mismo tamaño del dataset original para después apendearlo

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc #garbage collector

In [2]:
!ls ../data/netflix-prize-data

README
combined_data_1.txt
combined_data_1_with_movie_id.csv
combined_data_2.txt
combined_data_3.txt
combined_data_4.txt
movie_titles.csv
probe.txt
qualifying.txt


In [3]:
import chardet
with open("../data/netflix-prize-data/combined_data_1.txt", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(1000))
    
print(result)

{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [4]:
netflix = "..//data//netflix-prize-data//"

In [5]:
%%time
def load_data(name):
    df = pd.read_csv(name, header = None, names = ['User','Rating'], usecols = [0,1])
    return df

df1 = load_data(netflix + 'combined_data_1.txt')
print(df1.shape)

(24058263, 2)
Wall time: 17.7 s


In [6]:
df1.head()

,User,Rating
0,1:,NaN
1,1488844,3.0
2,822109,5.0
3,885013,4.0
4,30878,4.0


In [7]:
df1.Rating.isna()

0            True
1           False
2           False
3           False
4           False
            ...  
24058258    False
24058259    False
24058260    False
24058261    False
24058262    False
Name: Rating, Length: 24058263, dtype: bool

In [8]:
len(df1[df1.Rating.isna()==True])

4499

In [9]:
df1.User.values

array(['1:', '1488844', '822109', ..., '512536', '988963', '1704416'],
      dtype=object)

In [10]:
print(len(df1.User[df1.Rating.isna()].values))

4499


## Titulo-ID

In [11]:
df_title = pd.read_csv('../data/netflix-prize-data/movie_titles.csv', encoding = "ISO-8859-1",index_col = 0, header = None, usecols = [0,2], names = ['Movie_Id', 'Name'])
df_title.head()

,Name
Movie_Id,
1,Dinosaur Planet
2,Isle of Man TT 2004 Review
3,Character
4,Paula Abdul's Get Up & Dance
5,The Rise and Fall of ECW


In [12]:
movie_id = 1
print(df_title.loc[movie_id].Name)

Dinosaur Planet


### Agrego el movieId

In [13]:
#notar en el head que el id de nivue esta cómo 1: y rating dice nan y el resto son ids de peliculas si no tiene : o rating no tiene nan
movies_ids_df1 = df1.User[df1.Rating.isna()].values #me quedo con los nan como id de las movies
print(movies_ids_df1)
print(len(movies_ids_df1))

['1:' '2:' '3:' ... '4497:' '4498:' '4499:']
4499


In [14]:
#cambio el formato con numpy pasandolo cómo correspondiente al valor de index+1
movies_ids_df1 = np.arange(1,len(movies_ids_df1) + 1)
print(movies_ids_df1) #ahora tengo las ids de las movies y luego el listado de los usuarios

[   1    2    3 ... 4497 4498 4499]


In [15]:
##############################-----------------************* todo este bloque se hace solo para sacar los idx_movies de abajo
#fijate que filtras los nan del rating nada más
df1_nan = pd.DataFrame(pd.isnull(df1.Rating)) #condicion pd.isnull , fijate que arriba lo de values lo hice solo para mostrarte y bueno definir que era el idmovies
df1_nan = df1_nan[df1_nan['Rating'] == True] #quiero entonces trabajar sobre el datase df1

In [16]:
idx_movies_ids = df1_nan.index.values
print(idx_movies_ids) #defino los valores nulos de rating como el indice del filtro df1_nan

[       0      548      694 ... 24056849 24057564 24057834]


In [17]:
idx_movies_ids = np.append(idx_movies_ids,df1.shape[0]) #al apendearlo
cantidad_criticas = np.diff(idx_movies_ids) #puedo tener la diferencia para contar??
cantidad_criticas

array([ 548,  146, 2013, ...,  715,  270,  429], dtype=int64)

In [20]:
##estoy construyendo la matriz cantidad de criticas por película
columna_movie_id = np.array([])
for i in range(cantidad_criticas.size): #size cuenta las peliculas
    aux = np.full(cantidad_criticas[i], movies_ids_df1[i]) #np.full crea una matriz compuesta entre cantidad de peliculas y movie id
    columna_movie_id = np.concatenate((columna_movie_id, aux))

In [24]:
cantidad_criticas[1]

146

In [26]:
movies_ids_df1[1]

2

In [28]:
columna_movie_id[1]

1.0

In [37]:
df1['Movie_id'] = columna_movie_id
df1

,User,Rating,Movie_id
0,1:,NaN,1.0
1,1488844,3.0,1.0
2,822109,5.0,1.0
3,885013,4.0,1.0
4,30878,4.0,1.0
...,...,...,...
24058258,2591364,2.0,4499.0
24058259,1791000,2.0,4499.0
24058260,512536,5.0,4499.0
24058261,988963,3.0,4499.0


In [ ]:
df1.to_csv(netflix + 'combined_data_1_with_movie_id.csv', index= False)